In [1]:
import os

In [2]:
%pwd

'C:\\Users\\peguy\\PycharmProjects\\kidneydiseaseklassification\\research'

In [3]:
os.chdir('..')

In [4]:
%pwd

'C:\\Users\\peguy\\PycharmProjects\\kidneydiseaseklassification'